---
title: Visualize circuit timing
description: Visualize the timing of generated circuits by generating a figure that you can view, download, or both.
---


{/* cspell:ignore mactex, backgroundcolor, lightgreen */}

# Visualize circuit timing

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.3.0
qiskit-ibm-runtime~=0.43.1
```
</details>

In addition to [visualizing instructions on a circuit](/docs/guides/visualize-circuits), you might want to visualize a circuit's scheduling by using the Qiskit [`timeline_drawer`](/docs/api/qiskit/qiskit.visualization.timeline_drawer) method. This visualization could help you to quickly spot idling time on qubits, for example. However, this method does not return accurate results for dynamic circuits.  To visualize dynamic circuit scheduling, use the `draw_circuit_schedule_timing` method, as described in the [Qiskit Runtime support](#qr-support) section.

## Examples

To visualize a scheduled circuit program, you can call this function with a set of control arguments. Most of the  output image's appearance can be modified by a stylesheet, but this is not required.

### Draw with the default stylesheet

In [1]:
from qiskit import QuantumCircuit
from qiskit.visualization.timeline import draw
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.transpiler import generate_preset_pass_manager

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

backend = GenericBackendV2(5)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

draw(isa_circuit, target=backend.target)

<Image src="/docs/images/guides/visualize-circuit-timing/extracted-outputs/5b21b8c1-0.svg" alt="Output of the previous code cell" />

### Draw with a stylesheet suited for program debugging

In [2]:
from qiskit import QuantumCircuit
from qiskit.visualization.timeline import draw, IQXDebugging
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.transpiler import generate_preset_pass_manager

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

backend = GenericBackendV2(5)
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)
draw(isa_circuit, style=IQXDebugging(), target=backend.target)

<Image src="/docs/images/guides/visualize-circuit-timing/extracted-outputs/907dc46c-0.svg" alt="Output of the previous code cell" />

You can create custom generator or layout functions and update an existing stylesheet with the custom functions. This way, you can control most of the appearance of the output image without modifying the codebase of the scheduled circuit drawer.  See the [`timeline_drawer`](/docs/api/qiskit/qiskit.visualization.timeline_drawer) API reference for more examples.

<span id="qr-support"></span>
## Qiskit Runtime support

While the timeline drawer built in to Qiskit is useful for static circuits, it might not accurately reflect the timing of [dynamic circuits](/docs/guides/classical-feedforward-and-control-flow) because of implicit operations such as broadcasting and branch determination. As part of dynamic circuit support, Qiskit Runtime returns the accurate circuit timing information inside the job results when requested.

<Admonition type="note" title="Notes">
- This is an experimental function. It is in preview release status and is therefore subject to change.
- This function only applies to Qiskit Runtime Sampler jobs.
- Although the total circuit time is returned in the "compilation" metadata, this is NOT the time used for billing (quantum time).
</Admonition>

### Enable timing data retrieval

To enable timing data retrieval, set the experimental `scheduler_timing` flag to `True` when running the primitive job.

In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

sampler = SamplerV2(backend)
sampler.options.experimental = {
    "execution": {
        "scheduler_timing": True,
    },
}

sampler_job = sampler.run([isa_circuit])
result = sampler_job.result()

### Access the circuit timing data

When requested, the circuit timing data for each PUB is returned in the job result metadata, under `["compilation"]["scheduler_timing"]["timing"]`. This field contains the raw timing information. To display the timing information, use the built-in visualization tool, as described in the [Visualize the timings](#visualize-timings) section.

Use the following code to access the circuit timing data for the first PUB:

In [4]:
job_result = sampler_job.result()
circuit_schedule = job_result[0].metadata["compilation"]["scheduler_timing"]
circuit_schedule_timing = circuit_schedule["timing"]

#### Understand the raw timing data

While visualizing the circuit timing data by using the `draw_circuit_schedule_timing` method is the most common use case, it might be useful to understand the structure of the raw timing data returned. This could help you, for example, to extract information programmatically.

The timing data returned in `["compilation"]["scheduler_timing"]["timing"]` is a list of strings. Each string represents a single instruction on some channel and is comma-separated into the following data types:

- `Branch` - Determines whether the instruction is in a control flow (then / else) or a main branch.
- `Instruction` - The gate and the qubit to operate on.
- `Channel` - The channel that is being assigned with the instruction. It can be one of the following:
      - `Qubit x` - The drive channel for qubit _x_.
      - `AWGRx_y` (arbitrary waveform generator readout) - Used by readout channels to communicate when measuring qubits. The _x_ and _y_ arguments correspond to the readout instrument ID and the qubit number, respectively.
- `T0` - The instruction start time within the complete schedule
- `Duration` - The instruction's duration, in units of _dt_ seconds,  where 1 dt = 1 scheduling cycle. You can find the `dt` value of a backend by using [`backend.dt`](/docs/api/qiskit/qiskit.providers.BackendV2#dt).
- `Pulse` - The type of pulse operation being used.


Example:

```python
main,barrier,Qubit 0,7,0,barrier # A barrier on the main branch on qubit 0 at time 7 with 0 duration
main,reset_0,Qubit 0,7,64,play # A reset instruction on the main branch on qubit 0 at time 7 with duration 64 and a play operation
...
```

<span id="visualize-timings"></span>
### Visualize the timings

With `qiskit-ibm-runtime` v0.43.0 or later, you can visualize circuit timings. To visualize the timings, you first need to convert the result metadata to `fig` by using the [`draw_circuit_schedule_timing` method.](https://github.com/Qiskit/qiskit-ibm-runtime/blob/3d1bf1e1d49e5123841639fce259859c90ce9314/qiskit_ibm_runtime/visualization/draw_circuit_schedule_timings.py#L26) This method returns a `plotly` figure, which you can display directly, save to a file, or both.  For more information about the `plotly` commands to use, see [`fig.show()`](https://plotly.com/python-api-reference/generated/plotly.io.show.html) and  [`fig.write_image("<path.format>")`.](https://plotly.com/python-api-reference/generated/plotly.io.write_image.html)

In [5]:
from qiskit_ibm_runtime.visualization import draw_circuit_schedule_timing

# Create a figure from the metadata
fig = draw_circuit_schedule_timing(
    circuit_schedule=circuit_schedule_timing,
    included_channels=None,
    filter_readout_channels=False,
    filter_barriers=False,
    width=1000,
)

# Uncomment the following line to display the figure
# fig.show(renderer="notebook")

# Save to a file
# fig.write_html("scheduler_timing.html")

![Hovering over the output shows information such as the start, finish, and duration.](/docs/images/guides/visualize-circuit-timing/image_1.svg 'Example of a generated figure')

#### Understand the generated figure

The image of the circuit timing data output by `draw_circuit_schedule_timing` conveys the following information:

- X axis is time in units of _dt_ seconds,  where 1 dt = 1 scheduling cycle. You can find the `dt` value of a backend by using [`backend.dt`](/docs/api/qiskit/qiskit.providers.BackendV2#dt).
- Y axis is the channel (think of channels as instruments that emit pulses).
    - `Receive channel` - The only channel that isn't an instrument by itself. It is an instruction played on all channels that are part of a communication procedure with the hub at that time.
    - `Qubit x` - The drive channel for qubit x.
    - `AWGRx_y` (arbitrary waveform generator readout) - Used by readout channels to communicate when measuring qubits. The _x_ and _y_ arguments correspond to the readout instrument ID and the qubit number, respectively.
    - `Hub` - Controls broadcasting.

Additionally, each instruction has the format of *X_Y*, where *X* is the name of the instruction and *Y* is the pulse type. A `play` type applies control pulses, and a `capture` records the qubit's state. You can also hover over each instruction to get more details. For example, the previous figure shows a control pulse for the X gate applied to qubit 10 at 1161 dt.

### End-to-end example

This example shows you how to enable the option, get the circuit timing information from the metadata, and display it as an image.

First, set up the environment, define the circuits and convert them to ISA circuits, and define and run the jobs.

In [6]:
from qiskit_ibm_runtime import SamplerV2, QiskitRuntimeService
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.transpiler import generate_preset_pass_manager

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Create a dynamic circuit

qubits = QuantumRegister(1)
clbits = ClassicalRegister(1)
qc = QuantumCircuit(qubits, clbits)
(q0,) = qubits
(c0,) = clbits

qc.h(q0)
qc.measure(q0, c0)
with qc.if_test((c0, 1)):
    qc.x(q0)
qc.measure(q0, c0)

# Convert to an ISA circuit for the given backend

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

# Generate samplers for backend targets
sampler = SamplerV2(backend)
sampler.options.experimental = {"execution": {"scheduler_timing": True}}

# Submit jobs
sampler_job = sampler.run([isa_circuit])
result = sampler_job.result()

print(
    f">>> {' Job ID:':<10}  {sampler_job.job_id()} ({sampler_job.status()})"
)

>>>  Job ID:    d5kk3cn853es738e01dg (DONE)


Next, get the circuit schedule timing:

In [7]:
# Get the circuit schedule timing
result[0].metadata["compilation"]["scheduler_timing"]["timing"]

'main,rz_0,Qubit 0,929,0,shift_phase\nmain,sx_0,Qubit 0,929,8,play\nmain,sx_0,Qubit 0,933,0,shift_phase\nmain,rz_0,Qubit 0,937,0,shift_phase\nmain,barrier,Qubit 0,937,0,barrier\nmain,barrier,Qubit 0,937,0,barrier\nmain,barrier,Qubit 1,937,0,barrier\nmain,barrier,Qubit 2,937,0,barrier\nmain,barrier,Qubit 3,937,0,barrier\nmain,barrier,Qubit 4,937,0,barrier\nmain,barrier,Qubit 5,937,0,barrier\nmain,barrier,Qubit 6,937,0,barrier\nmain,barrier,Qubit 7,937,0,barrier\nmain,barrier,Qubit 8,937,0,barrier\nmain,barrier,Qubit 9,937,0,barrier\nmain,barrier,Qubit 10,937,0,barrier\nmain,barrier,Qubit 11,937,0,barrier\nmain,barrier,Qubit 12,937,0,barrier\nmain,barrier,Qubit 13,937,0,barrier\nmain,barrier,Qubit 14,937,0,barrier\nmain,barrier,Qubit 15,937,0,barrier\nmain,barrier,Qubit 16,937,0,barrier\nmain,barrier,Qubit 17,937,0,barrier\nmain,barrier,Qubit 18,937,0,barrier\nmain,barrier,Qubit 19,937,0,barrier\nmain,barrier,Qubit 20,937,0,barrier\nmain,barrier,Qubit 21,937,0,barrier\nmain,barrier,Qubit

Finally, you can visualize and save the timing:

In [8]:
from qiskit_ibm_runtime.visualization import draw_circuit_schedule_timing

circuit_schedule = result[0].metadata["compilation"]["scheduler_timing"][
    "timing"
]
fig = draw_circuit_schedule_timing(
    circuit_schedule=circuit_schedule,
    included_channels=None,
    filter_readout_channels=False,
    filter_barriers=False,
    width=1000,
)

# Uncomment the following line to display the figure
# fig.show(renderer="notebook")

# Save to a file
# fig.write_html("scheduler_timing.html")

## Next steps

<Admonition type="tip" title="Recommendations">
  - [Classical feedforward and control flow](/docs/guides/classical-feedforward-and-control-flow) (dynamic circuits)
  - [Visualize circuits](/docs/guides/visualize-circuits)
</Admonition>